In [1]:
# Importation des Librairies
import pandas as pd
import numpy as np

In [2]:
# PARTIE 15 ----------------------------------------------------------------
# Creation du dataframe
population_csv = pd.read_csv("/Users/teilo/Desktop/data/fr_population.csv")
population = pd.DataFrame(population_csv, columns=['Zone','Code zone', 'Année', 'Valeur'])

# Renommée des colonnes
nom_col = {'Code zone': 'code_pays', 'Année': 'annee', 'Zone': 'pays', 'Valeur': 'population'}
population.rename(columns=nom_col, inplace=True)

#Calcul des valeurs
population['population'] = population['population'] * 1000

# Suppression de la Chine
population[population['pays'] != 'Chine'].copy()

# Clé primaire
# code_pays et annee

# MySQL code
population.to_csv('population2.csv', encoding='utf-8')

population.head()

,pays,code_pays,annee,population
0,Afghanistan,2,2013,30552000
1,Afrique du Sud,202,2013,52776000
2,Albanie,3,2013,3173000
3,Algérie,4,2013,39208000
4,Allemagne,79,2013,82727000


In [3]:
# PARTIE 16 ----------------------------------------------------------------
# Création du dataframe
columns_df = ['Zone', 'Code zone', 'Année', 'Produit', 'Code Produit', 'Élément','Valeur']

# Dataframe des végétaux
veg_df = pd.read_csv("/Users/teilo/Desktop/data/fr_vegetaux.csv")[columns_df]
veg_df['origine'] = 'vegetal'

#Dataframe des animaux
anim_df = pd.read_csv("/Users/teilo/Desktop/data/fr_animaux.csv")[columns_df]
anim_df['origine'] = 'animal'

# Concatenation des DataFrames
liste_df = [veg_df, anim_df]
dispo_alim = pd.concat(liste_df)
dispo_alim.reset_index(drop=True, inplace=True)

#Pivot des tables
dispo_alim = pd.pivot_table(dispo_alim, index=['Zone', 'Code zone', 'Année', 'Produit', 'Code Produit', 'origine','Élément'], values='Valeur', columns='Élément', aggfunc= np.sum)
dispo_alim.reset_index(inplace=True)

#Renommée des colonnes
pre_columns = {'Zone': 'pays', 'Code zone': 'code_pays', 'Année': 'annee_alim', 'Code Produit': 'code_produit', 'Disponibilité alimentaire (Kcal/personne/jour)': 'dispo_alim_kcal','Disponibilité de matière grasse en quantité (g/personne/jour)': 'dispo_mat_gr','Disponibilité de protéines en quantité (g/personne/jour)':'dispo_prot','Disponibilité alimentaire en quantité (kg/personne/an)':'dispo_alim_tonnes'}
dispo_alim.rename(columns=pre_columns, inplace=True)

# Suppression de la Chine
dispo_alim = dispo_alim[dispo_alim['pays'] != 'Chine'].copy()



#Fusion des dataframes
temp_df = pd.DataFrame(population, columns=['annee','code_pays','population'])
dispo_alim = dispo_alim.merge(temp_df, how='left', left_on=['code_pays'], right_on=['code_pays'])

#Suppression des doublons
dispo_alim.drop_duplicates(inplace=True)

# Création d'une colonne pour les tonnes
dispo_alim['dispo_alim_tonnes'] = dispo_alim['dispo_alim_tonnes'] * dispo_alim['population']
#dispo_alim.drop(axis=1, labels=['dispo_alim_tonnes'], inplace= True)

dispo_alim2 = dispo_alim.copy()
# Nettoyage / RESET
dispo_alim = dispo_alim[['pays', 'code_pays', 'annee_alim', 'Produit','code_produit', 'origine', 'dispo_alim_tonnes', 'dispo_alim_kcal', 'dispo_prot', 'dispo_mat_gr', 'population']]
dispo_alim.fillna(0, inplace=True)

#Clé primaires
# code_pays et code_produit

dispo_alim.to_csv('dispo_alim2.csv', encoding='utf-8')

dispo_alim.head()

,pays,code_pays,annee_alim,Produit,code_produit,origine,dispo_alim_tonnes,dispo_alim_kcal,dispo_prot,dispo_mat_gr,population
0,Afghanistan,2,2013,Abats Comestible,2736,animal,0.0,5.0,0.00,0.0,30552000
1,Afghanistan,2,2013,Abats Comestible,2736,animal,52549440.0,0.0,0.00,0.0,30552000
2,Afghanistan,2,2013,Abats Comestible,2736,animal,0.0,0.0,0.00,0.2,30552000
3,Afghanistan,2,2013,Abats Comestible,2736,animal,0.0,0.0,0.77,0.0,30552000
4,Afghanistan,2,2013,Abats Comestible,2736,animal,0.0,0.0,0.00,0.0,30552000


In [4]:
#PARTIE 17 ----------------------------------------------------------------
# Création du dataframe
equilibre_prod = dispo_alim2.copy()


equilibre_prod = pd.pivot_table(equilibre_prod, index=['pays','code_pays', 'code_produit', 'annee', 'Produit'], aggfunc= np.sum,)
equilibre_prod.reset_index(inplace=True)

# Renommée des colonnes
equi_col = {'Aliments pour animaux' : 'alim_anim', 'Autres utilisations (non alimentaire)': 'autres_usages', 'Disponibilité intérieure': 'dispo_int'}
equilibre_prod.rename(columns=equi_col, inplace=True)
equilibre_prod.fillna(0, inplace=True)

#equilibre_prod = equilibre_prod[equilibre_prod['Nourriture']]
#equilibre_prod = equilibre_prod[equilibre_prod['alim_anim']]
equilibre_prod = equilibre_prod[equilibre_prod['annee'] == 2013]



# Suppression de la Chine
equilibre_prod = equilibre_prod[equilibre_prod['pays'] != 'Chine'].copy()

#Merge des DF
equilibre_prod = equilibre_prod[['pays', 'code_pays','annee','Produit', 'code_produit','dispo_int','alim_anim', 'Semences', 'Pertes', 'Traitement','Nourriture', 'autres_usages']]
#Clé primaire
# code_pays, code_produit et annee (si changé)

equilibre_prod.to_csv('equilibre_prod2.csv', encoding='utf-8')
equilibre_prod.head()

,pays,code_pays,annee,Produit,code_produit,dispo_int,alim_anim,Semences,Pertes,Traitement,Nourriture,autres_usages
0,Afghanistan,2,2013,Blé,2511,5992.0,0.0,322.0,775.0,0.0,4895.0,0.0
1,Afghanistan,2,2013,Orge,2513,524.0,360.0,22.0,52.0,0.0,89.0,0.0
2,Afghanistan,2,2013,Maïs,2514,313.0,200.0,5.0,31.0,0.0,76.0,0.0
3,Afghanistan,2,2013,Millet,2517,13.0,0.0,0.0,1.0,0.0,12.0,0.0
4,Afghanistan,2,2013,"Céréales, Autres",2520,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
#PARTIE 18 ----------------------------------------------------------------------------
# Création du dataframe
sous_nutrition = pd.read_csv("/Users/teilo/Desktop/data/fr_sousalimentation.csv")[['Zone','Code zone', 'Code année', 'Valeur']]

# Renommée des colonnes
sous_nut_col = {'Zone': 'pays', 'Code zone': 'code_pays', 'Code année': 'annee', 'Valeur': 'nb_personnes'}
sous_nutrition.rename(columns=sous_nut_col, inplace=True)

# Nettoyage / RESET
sous_nutrition['nb_personnes'] = sous_nutrition['nb_personnes'].replace({'<0.1' : '0'})
sous_nutrition.fillna(0, inplace=True)
# sous_nutrition['nb_personnes'] = sous_nutrition['nb_personnes'] * 1000000
sous_nutrition['nb_personnes'] = sous_nutrition['nb_personnes'].astype(float)
sous_nutrition['nb_personnes'] = sous_nutrition['nb_personnes'] * 1000000
# Suppression de la Chine
sous_nutrition = sous_nutrition[sous_nutrition['pays'] != 'Chine'].copy()

# Formatage des dates
def format_annee(code_annee):


    if code_annee == 20112013:
        return 2012

    elif code_annee == 20122014:
        return 2013

    elif code_annee == 20132015:
        return 2014

    elif code_annee == 20142016:
        return 2015

    elif code_annee == 20152017:
        return 2016

    elif code_annee == 20162018:
        return 2017


sous_nutrition['annee'] = sous_nutrition['annee'].apply(format_annee)

sous_nutrition = sous_nutrition[sous_nutrition['annee'] == 2013]

# Clé primaires
# code_pays

sous_nutrition.to_csv('sous_nutrition2.csv', encoding='utf-8')
sous_nutrition.head()

,pays,code_pays,annee,nb_personnes
0,Afghanistan,2,2013,7900000.0
5,Afrique du Sud,202,2013,2600000.0
10,Albanie,3,2013,200000.0
15,Algérie,4,2013,1700000.0
20,Allemagne,79,2013,0.0


In [6]:
#PARTIE 19 ----------------------------------------------------------------
# Création du dataframe
dispo_alim_hab = pd.DataFrame(columns={'annee', 'pays', 'dispo_alim_kcal_p_j'})

dispo_alim_hab.to_csv('dispo_alim_hab2.csv', encoding='utf-8')

dispo_alim_hab = pd.DataFrame(columns={'annee', 'pays', 'dispo_prot'})

dispo_alim_hab.to_csv('dispo_alim_hab2.csv', encoding='utf-8')

In [7]:
#PARTIE 19 n1 ---------------------------------
  # 19   n1 TOP 10 prot/kcal // habitant  
    
# Création du dataframe    
top_alim = dispo_alim[['pays', 'code_pays', 'annee_alim', 'dispo_prot', 'dispo_alim_kcal', 'population']]

#Remise à l'année
top_alim['dispo_prot_par_humain_kg'] = top_alim['dispo_prot'] * 365 * 0.001
top_alim['dispo_kcal_par_humain'] = top_alim['dispo_alim_kcal'] * 365
top_alim['dispo_prot_par_jour'] = top_alim['dispo_prot']
top_alim['dispo_kcal_par_jour'] = top_alim['dispo_alim_kcal']

#Groupby par pays sur les disponibilités
top_alim = top_alim[['pays', 'dispo_prot_par_jour', 'dispo_kcal_par_jour', 'dispo_prot_par_humain_kg', 'dispo_kcal_par_humain']].groupby('pays').sum()

top_alim.head()

/var/folders/c8/_qv654q539994nw7lkc47kgh0000gn/T/ipykernel_4425/4221677163.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_alim['dispo_prot_par_humain_kg'] = top_alim['dispo_prot'] * 365 * 0.001
/var/folders/c8/_qv654q539994nw7lkc47kgh0000gn/T/ipykernel_4425/4221677163.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_alim['dispo_kcal_par_humain'] = top_alim['dispo_alim_kcal'] * 365
/var/folders/c8/_qv654q539994nw7lkc47kgh0000gn/T/ipykernel_4425/4221677163.py:10: SettingWithCopyWarning: 
A val

,dispo_prot_par_jour,dispo_kcal_par_jour,dispo_prot_par_humain_kg,dispo_kcal_par_humain
pays,,,,
Afghanistan,58.26,2087.0,21.26490,761755.0
Afrique du Sud,85.33,3020.0,31.14545,1102300.0
Albanie,111.37,3188.0,40.65005,1163620.0
Algérie,91.92,3293.0,33.55080,1201945.0
Allemagne,101.39,3503.0,37.00735,1278595.0


In [8]:
top_alim.sort_values('dispo_prot_par_humain_kg', ascending=False)[:10]

,dispo_prot_par_jour,dispo_kcal_par_jour,dispo_prot_par_humain_kg,dispo_kcal_par_humain
pays,,,,
Islande,133.06,3381.0,48.56690,1234065.0
Chine - RAS de Hong-Kong,129.07,3286.0,47.11055,1199390.0
Israël,128.00,3610.0,46.72000,1317650.0
Lituanie,124.36,3414.0,45.39140,1246110.0
Maldives,122.32,2733.0,44.64680,997545.0
Finlande,117.56,3366.0,42.90940,1228590.0
Luxembourg,113.64,3540.0,41.47860,1292100.0
Monténégro,111.90,3490.0,40.84350,1273850.0
Pays-Bas,111.46,3222.0,40.68290,1176030.0


In [9]:
top_alim.sort_values('dispo_kcal_par_humain', ascending=False)[:10]

,dispo_prot_par_jour,dispo_kcal_par_jour,dispo_prot_par_humain_kg,dispo_kcal_par_humain
pays,,,,
Autriche,106.20,3770.0,38.76300,1376050.0
Belgique,99.37,3737.0,36.27005,1364005.0
Turquie,108.12,3708.0,39.46380,1353420.0
États-Unis d'Amérique,109.42,3682.0,39.93830,1343930.0
Israël,128.00,3610.0,46.72000,1317650.0
Irlande,109.91,3602.0,40.11715,1314730.0
Italie,108.49,3578.0,39.59885,1305970.0
Luxembourg,113.64,3540.0,41.47860,1292100.0
Égypte,103.22,3518.0,37.67530,1284070.0


In [10]:
#q19 n2 ---------
        # Bottom 10 dispo prot
top_alim.sort_values('dispo_prot_par_humain_kg', ascending=True)[:10]

,dispo_prot_par_jour,dispo_kcal_par_jour,dispo_prot_par_humain_kg,dispo_kcal_par_humain
pays,,,,
Libéria,37.66,2206.0,13.74590,805190.0
Guinée-Bissau,44.05,2288.0,16.07825,835120.0
Mozambique,45.68,2282.0,16.67320,832930.0
République centrafricaine,46.04,1879.0,16.80460,685835.0
Madagascar,46.69,2056.0,17.04185,750440.0
Haïti,47.70,2089.0,17.41050,762485.0
Zimbabwe,48.32,2113.0,17.63680,771245.0
Congo,51.41,2206.0,18.76465,805190.0
Ouganda,52.64,2126.0,19.21360,775990.0


In [11]:
#Q19 n3 --------------

#Création du df
pertes = equilibre_prod[['pays', 'Pertes']]

#Calcul de la somme des pertes
pertes['Pertes'].sum()

#Remises à la valeur
pertes['Pertes'] = pertes['Pertes'] * 1000000

pertes.sort_values('Pertes', ascending=False)[:10]

/var/folders/c8/_qv654q539994nw7lkc47kgh0000gn/T/ipykernel_4425/1530533051.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pertes['Pertes'] = pertes['Pertes'] * 1000000


,pays,Pertes
2091,Brésil,5.504700e+10
3320,"Chine, continentale",3.763800e+10
6195,Inde,1.133600e+10
3275,"Chine, continentale",1.025000e+10
2080,Brésil,8.145000e+09
3366,"Chine, continentale",6.393000e+09
9792,Nigéria,6.080000e+09
6187,Inde,5.611000e+09
6239,Inde,5.515000e+09
6200,Inde,5.118000e+09


In [12]:
# Q19 n4 ---------
    # TOP 10 pays en sous-nutrition

#Création du df en combinant les 2 tables    
sous_nut = pd.merge(population, sous_nutrition[['pays', 'code_pays', 'nb_personnes']], how='inner')
sous_nut.drop_duplicates()

#Calcul du ration et implementation
sous_nut['ratio_sous_nut'] = sous_nut['nb_personnes']/sous_nut['population']
sous_nut.sort_values('ratio_sous_nut', ascending=False)[:10]

,pays,code_pays,annee,population,nb_personnes,ratio_sous_nut
70,Haïti,93,2013,10317000,5200000.0,0.504022
172,Zambie,251,2013,14539000,7000000.0,0.481464
173,Zimbabwe,181,2013,14150000,6600000.0,0.466431
131,République centrafricaine,37,2013,4616000,2000000.0,0.433276
136,République populaire démocratique de Corée,116,2013,24895000,10600000.0,0.425788
39,Congo,46,2013,4448000,1800000.0,0.404676
157,Tchad,39,2013,12825000,4900000.0,0.382066
5,Angola,7,2013,21472000,8100000.0,0.377235
93,Libéria,123,2013,4294000,1600000.0,0.372613
97,Madagascar,129,2013,22925000,8200000.0,0.357688


In [13]:
# 19 n5 TOP 10 PRODUITS autres usages // dispo interieur
# Créations du dataframe
temp = equilibre_prod[['pays', 'Produit', 'dispo_int', 'autres_usages']]
temp = temp.groupby('Produit').sum()

#Calcul du ratio et implémentation
temp['ratio_autres_dispo'] = temp['autres_usages']/temp['dispo_int']
temp.sort_values('ratio_autres_dispo', ascending=False)[:10]

,dispo_int,autres_usages,ratio_autres_dispo
Produit,,,
"Alcool, non Comestible",21741.0,21769.0,1.001288
Huil Plantes Oleif Autr,6271.0,4733.0,0.754744
Huile de Palmistes,6570.0,4625.0,0.703957
Huile de Palme,50409.0,35191.0,0.698109
Girofles,130.0,84.0,0.646154
Huile de Colza&Moutarde,23416.0,12909.0,0.551290
Graisses Animales Crue,22951.0,10792.0,0.470219
Huiles de Poissons,828.0,371.0,0.448068
Huile de Soja,41961.0,17463.0,0.416172
